In [1]:
!pip install keras==2.12
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 47.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
import os
import h5py
import math
import numpy as np
import tensorflow as tf
import keras
import keras.utils
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import glob
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [6]:

from sklearn.model_selection import train_test_split

def create_tiles(image, tile_size):
    tiles = []
    img_height, img_width, _ = image.shape
    for i in range(0, img_height, tile_size):
        for j in range(0, img_width, tile_size):
            if i + tile_size <= img_height and j + tile_size <= img_width:
                tile = image[i:i+tile_size, j:j+tile_size]
                if tile.shape[0] == tile_size and tile.shape[1] == tile_size:
                    tiles.append(tile)
    return tiles

def create_tiles_from_folder(image_folder, tile_size, agbd_index, test_size=0.2, random_state=42):
    X = []
    y = []

    # Caricare e dividere le immagini in tiles
    for image_path in os.listdir(image_folder):
        if image_path.endswith('.tif'):
            with rasterio.open(os.path.join(image_folder, image_path)) as src:
                img = src.read().transpose((1, 2, 0))  # Convertire a formato (altezza, larghezza, canali)

                tiles = create_tiles(img, tile_size)

                for tile in tiles:
                    # Verifica se ci sono NaN nella tile
                    if not np.any(np.isnan(tile)):
                        tile[np.isnan(tile)] =0  # Sostituisci NaN con -1

                        # Aggiungi la tile a X (senza l'ultimo canale) e a y (solo l'ultimo canale)
                        X.append(tile[..., :agbd_index])
                        y.append(tile[..., agbd_index])

    # Converti in array NumPy
    X = np.array(X)
    y = np.array(y)

    # Verifica delle dimensioni
    assert X.shape[0] == y.shape[0], f"Numero di campioni in X ({X.shape[0]}) e y ({y.shape[0]}) non corrisponde"

    # Suddividi il dataset in training e test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test



# Parametri di esempio
image_folder = '/content/drive/MyDrive/immagini_nuovo'
tile_size = 64  # Dimensione del tile
agbd_index = 13  # Indice della banda AGBD

# Utilizzo della funzione per ottenere X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = create_tiles_from_folder(image_folder, tile_size, agbd_index)


# Stampa delle dimensioni per verificare
print(f"Dimensioni di X_train: {X_train.shape}")
print(f"Dimensioni di X_test: {X_test.shape}")
print(f"Dimensioni di y_train: {y_train.shape}")
print(f"Dimensioni di y_test: {y_test.shape}")


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [12]:
import os
import numpy as np
import rasterio
from sklearn.model_selection import train_test_split

def create_tiles(image, tile_size):
    tiles = []
    img_height, img_width, _ = image.shape
    for i in range(0, img_height, tile_size):
        for j in range(0, img_width, tile_size):
            if i + tile_size <= img_height and j + tile_size <= img_width:
                tile = image[i:i+tile_size, j:j+tile_size]
                if tile.shape[0] == tile_size and tile.shape[1] == tile_size:
                    tiles.append(tile)
    return tiles

def create_tiles_from_folder(image_folder, tile_size, agbd_index, test_size=0.2, random_state=42):
    X = []
    y = []

    # Caricare e dividere le immagini in tiles
    for image_path in os.listdir(image_folder):
        if image_path.endswith('.tif'):
            with rasterio.open(os.path.join(image_folder, image_path)) as src:
                img = src.read().transpose((1, 2, 0))  # Convertire a formato (altezza, larghezza, canali)

                tiles = create_tiles(img, tile_size)

                for tile in tiles:
                    # Verifica se ci sono NaN nella tile
                    if not np.any(np.isnan(tile)):
                        tile[np.isnan(tile)] = 0  # Sostituisci NaN con -1

                        # Aggiungi la tile a X (senza l'ultimo canale) e a y (solo l'ultimo canale)
                        X.append(tile[..., :agbd_index])
                        y.append(tile[..., agbd_index])

    # Converti in array NumPy
    X = np.array(X)
    y = np.array(y)

    # Verifica delle dimensioni
    assert X.shape[0] == y.shape[0], f"Numero di campioni in X ({X.shape[0]}) e y ({y.shape[0]}) non corrisponde"


    # Suddividi il dataset in training e test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test


# Parametri di esempio
image_folder = '/content/drive/MyDrive/immagini_nuovo'
tile_size = 64  # Dimensione del tile
agbd_index = 13  # Indice della banda AGBD

# Utilizzo della funzione per ottenere X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = create_tiles_from_folder(image_folder, tile_size, agbd_index)

# Stampa delle dimensioni per verificare
print(f"Dimensioni di X_train: {X_train.shape}")
print(f"Dimensioni di X_test: {X_test.shape}")
print(f"Dimensioni di y_train: {y_train.shape}")
print(f"Dimensioni di y_test: {y_test.shape}")


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
np.save('x_train.npy', X_train)
np.save('x_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

In [ ]:
def remove_invalid_tiles(X, y):
    # Crea una maschera per i tiles che non contengono valori non validi (ad esempio, -1)
    mask_X = ~np.any(X == 0, axis=(1, 2, 3))
    mask_y = ~np.any(y == 0, axis=(1, 2, 3))

    # Combina le due maschere
    mask = mask_X & mask_y

    # Applica la maschera a X e y
    X_filtered = X[mask_X]
    y_filtered = y[mask_y]

    return X_filtered, y_filtered

In [ ]:
x_train = np.load('/content/drive/MyDrive/x_train.npy')
x_test = np.load('/content/drive/MyDrive/x_test.npy')
y_train = np.load('/content/drive/MyDrive/y_train.npy')
y_test = np.load('/content/drive/MyDrive/y_test.npy')

In [ ]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

x_train shape: (13528, 64, 64, 13)
y_train shape: (13528, 64, 64)


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
import tensorflow.keras.backend as K

def identity_block(X, f, filters, stage, block):
    conv_base_name = 'res' + str(stage) + block + '_'
    bn_base_name = 'bn' + str(stage) + block + '_'

    f1, f2, f3 = filters
    bn_axis = 3
    X_skip_connection = X

    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_base_name + 'first_component', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=bn_axis, name=bn_base_name + 'first_component')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=f2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_base_name + 'second_component', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=bn_axis, name=bn_base_name + 'second_component')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_base_name + 'third_component', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=bn_axis, name=bn_base_name + 'third_component')(X)

    X = Add()([X, X_skip_connection])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s=2):
    F1, F2, F3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape=(64, 64, 13)):
    X_input = Input(input_shape)

    X = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1', kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')


    X = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same', name='deconv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_deconv1')(X)
    X = Activation('relu')(X)

    X = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', name='deconv2', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_deconv2')(X)
    X = Activation('relu')(X)

    X = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', name='deconv3', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_deconv3')(X)
    X = Activation('relu')(X)

    X = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', name='deconv4', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_deconv4')(X)
    X = Activation('relu')(X)


    X = Conv2D(1, (1, 1), activation='linear', name='conv_final', kernel_initializer=glorot_uniform(seed=0))(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


model = ResNet50(input_shape=(64, 64, 13))
model.compile(optimizer='adam', loss='mse',metrics='mae')


model.summary()


history = model.fit(x_train, y_train, epochs=20, batch_size=32)


Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64, 64, 13)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1 (Conv2D)                 (None, 32, 32, 64)   40832       ['input_3[0][0]']                
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1[0][0]']                  
                                                                                                  
 activation_106 (Activation)    (None, 32, 32, 64)   0           ['bn_conv1[0][0]']        

In [ ]:
model.save('/content/drive/MyDrive/agb_2.h5')

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/agb_2.h5')

In [ ]:
test_evaluate = new_model.evaluate(x_test)
print("Loss: {}".format(test_evaluate[0]))

106/106 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Loss: 0.0


In [ ]:
predictions = new_model.evaluate(x_test, y_test)
print(predictions)

106/106 [==============================] - 350s 3s/step - loss: 153.3060 - mae: 2.8910
[153.30599975585938, 2.8909919261932373]


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Ottieni le predizioni del modello
predictions = new_model.evaluate(x_test,y_test)

# Calcola l'MSE (Mean Squared Error)
mse = mean_squared_error(y_test, predictions)

# Calcola l'RMSE prendendo la radice quadrata dell'MSE
rmse = np.sqrt(mse)

print("Root Mean Squared Error (RMSE):", rmse)
